In [ ]:
#install the "datasets" and "transformers" libraries
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 12.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# required libraries 
import torch
import json
import progressbar
import argparse
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F
import matplotlib
import matplotlib.pyplot as plt
import torch.nn.functional as F
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
#function loads a pre-trained language model and tokenize
def load_model_tokenizer(model_name):
    return AutoModel.from_pretrained(model_name), AutoTokenizer.from_pretrained(model_name)


In [ ]:
#generates a diagonal mask matrix
def get_diag_mask_matrix(seqlen):
    m= np.zeros((seqlen, seqlen), float)
    np.fill_diagonal(m, 1.0)
    return 1.0 - m

In [ ]:
#tokenizes a sentence using a given tokenizer object
def text_to_id(sentence, tokenizer, is_cuda, device):
    sentence = '[CLS] ' + sentence + ' [SEP]'
    token_id = torch.LongTensor(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence) )).view(1,-1)
    if is_cuda:
        token_id = token_id.cuda(device)
    return token_id


In [ ]:
def compute_cosine_correlation_matrix(hidden_tensor):
    # retrieve the second dimension of hidden tensor to get the sequence length
    _, seq_leng, _ = hidden_tensor.size()
    # normalize hidden tensor along last dimension using PyTorch's norm method, divide each vector by its norm
    norm_hidden = hidden_tensor / hidden_tensor.norm(dim=2, keepdim=True) 
    # take transpose of normalized hidden tensor
    norm_hidden_transpose = norm_hidden.transpose(1, 2)
    # compute cosine similarity between each pair of vectors in the original tensor using dot product of normalized tensor and its transpose
    correlation_matrix = torch.matmul(norm_hidden, norm_hidden_transpose).view(seq_leng, seq_leng)
    # convert correlation matrix to NumPy array, detach from computation graph, and return
    return correlation_matrix.detach().cpu().numpy()


In [ ]:

#function calculates the sum of cosine similarities

def compute_stat(hidden):
    # Compute the cosine similarity matrix for the input tensor
    cosine_correlation_matrix = compute_cosine_correlation_matrix(hidden)
    
    # Create a diagonal mask matrix to exclude diagonal elements from the sum of cosine similarities,Get the length of the sequence from the cosine similarity matrix
    mask_matrix = get_diag_mask_matrix(cosine_correlation_matrix.shape[0])
    
    # Check that the mask matrix has the same shape as the cosine similarity matrix
    assert mask_matrix.shape == cosine_correlation_matrix.shape
    
    # Apply the mask matrix to the cosine similarity matrix to exclude diagonal elements
    cosine_masked_matrix = cosine_correlation_matrix * mask_matrix

    #length of the sequence from the cosine similarity matrix
    seq_len  = cosine_correlation_matrix.shape[0]

    # Calculate the sum of cosine similarities between pairs of vectors, excluding diagonal elementstotal number of pairs of vectors in the sequence and return.
    return np.sum(cosine_masked_matrix) / 2, seq_len * (seq_len - 1) / 2


In [ ]:
#function processes a single text instance using a language model and computes the cosine similarity sum and the total number of token pairs for each attention layer of the model
def process_one_instance(model, tokenizer, text, is_cuda, device):
    # initialize an empty dictionary to store the cosine similarity sum and the total number of token pairs for each attention layer
    res_dic = {}
    for id in range(1,13):
        # add an empty dictionary for each attention layer to the result dictionary
        res_dic[id] = {}
    # tokenize the input text and convert it to a tensor of token IDs
    input_id = text_to_id(text, tokenizer, is_cuda, device)
    # run the input through the language model and retrieve the hidden states for each attention layer
    outputs = model(input_id, output_hidden_states=True)
    attention_hidden_states = outputs[-1]
    for id in range(1,13):
        # compute the cosine similarity sum and the total number of token pairs for the current attention layer
        one_cosine_sum, one_token_num = compute_stat(attention_hidden_states[id])
        # add the cosine similarity sum and the total number of token pairs to the result dictionary for the current attention layer
        res_dic[id]['cosine_sum'] = one_cosine_sum
        res_dic[id]['token_sum'] = one_token_num
    # return the result dictionary
    return res_dic


In [ ]:
# Function to parse command line arguments
def parse_config():
    # Create an argument parser
    parser = argparse.ArgumentParser()
    # Add three arguments to the parser: model_name, file_path, and output_path, all of type string
    parser.add_argument("--model_name", type=str)
    parser.add_argument("--file_path", type=str)
    parser.add_argument("--output_path", type=str)
    # Parse the arguments from the command line and return them as a Namespace object
    return parser.parse_args()


In [ ]:
#function is a wrapper function that performs layer-wise intra-sentence similarity analysis on a text file containing sentence

all_res_dict = {} # create an empty dictionary to store the output for each input text

def startAnalysis(model_name, file_path, output_path):
    # create an argument parser to take command-line arguments for model name, input file path, and output file path
    import argparse
    parser = argparse.ArgumentParser(description='Calculate layer-wise intra-sentence similarity using a BERT model.')
    parser.add_argument('--model_name', type=str, required=True, help='Name or path of the pre-trained BERT model')
    parser.add_argument('--file_path', type=str, required=True, help='Path to the text file containing sentences for similarity calculation')
    parser.add_argument('--output_path', type=str, required=True, help='Path to save the output JSON file')
    args = parser.parse_args([f"--model_name={model_name}", f"--file_path={file_path}", f"--output_path={output_path}"])

    # check if GPU is available
    if torch.cuda.is_available():
        print ('Cuda is available.')
    is_cuda = torch.cuda.is_available()

    # set the device to CUDA if available, otherwise use CPU
    device = torch.device('cuda') if is_cuda else torch.device('cpu')

    # load the specified pre-trained BERT model and tokenizer
    model, tokenizer = load_model_tokenizer(args.model_name)

    # move the model to CUDA if available
    if is_cuda:
        model = model.cuda(device)

    # set the model to evaluation mode
    model.eval()

    # read input text from the file and store in a list
    text_list = []
    with open(args.file_path, 'r', encoding='utf8') as i:
        lines = i.readlines()
        for l in lines:
            text_list.append(l.strip('\n'))

    # use progressbar to track progress while processing input texts
    import progressbar
    print('Start measuring intra-sentence similarity...')
    print('Number of text is {}'.format(len(text_list)))
    p = progressbar.ProgressBar(len(text_list))
    p.start()

    # loop over all input texts
    for idx in range(len(text_list)):
        p.update(idx)
        one_text = text_list[idx]
        one_res_dict = process_one_instance(model, tokenizer, one_text, is_cuda, device) # get the output for one input text
        all_res_dict[idx] = one_res_dict # store the output in the all_res_dict dictionary

    p.finish()
    print('Completed !')


In [ ]:
cd drive/MyDrive/NLP/TaCL-main/analysis

/content/drive/MyDrive/NLP/TaCL-main/analysis


In [ ]:
#The startAnalysis function will load the specified TACL BERT model and tokenizer, 
#read in the text file, and then process each sentence in the file using the process_one_instance function. 
#The results of the analysis will be saved to the specified output file in JSON form
startAnalysis(
    model_name='cambridgeltl/tacl-bert-base-uncased',
    file_path='zh_wiki_randomly_select_50k.txt',
    output_path='./tacl_result-2.json'
)
#dictionary into a JSON file
with open('./tacl_resul.json', 'w') as outfile:
      json.dump(all_res_dict, outfile, indent=4)


/ |#                                              | 50000 Elapsed Time: 0:00:00

Start measuring intra-sentence similarity...
Number of text is 50000


| |                                            #  | 49999 Elapsed Time: 3:34:24


Completed !


In [ ]:
#The startAnalysis function will load the specified Bert model and tokenizer, 
#read in the text file, and then process each sentence in the file using the process_one_instance function. 
#The results of the analysis will be saved to the specified output file in JSON form
startAnalysis(
    model_name='bert-base-uncased',
    file_path='./zh_wiki_randomly_select_50k.txt',
    output_path='./bert_resul.json'
)
#dictionary into a JSON file
with open('./bert_resul.json', 'w') as outfile:
      json.dump(all_res_dict, outfile, indent=4)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/ |#                                              | 50000 Elapsed Time: 0:00:00

Start measuring intra-sentence similarity...
Number of text is 50000


| |                             #                 | 49999 Elapsed Time: 3:06:41


Completed !


In [ ]:
#installs the latex Python package
!pip install latex
#Installs some required LATEX fonts and tools for creating PDFs from LATEX documents
!apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng

In [ ]:
#Function calculates average similarity score for each layer.
def process_result(in_f):
    # Load the result JSON file
    with open(in_f) as f:
        res_dict = json.load(f)
    # Print the number of instances
    print (len(res_dict))
    
    # Create a dictionary to store the cross-similarity result for each layer
    layer_res = {}
    for idx in range(1,13):
        layer_res[idx] = {'cosine_sum':0., 'token_sum':0}
    
    # Traverse each instance to update the cross-similarity result for each layer
    key_list = list(res_dict.keys())
    for key in key_list:
        instance = res_dict[key]
        for idx in range(1,13):
            key = str(idx)
            one_cosine_sum = instance[key]['cosine_sum']
            one_token_sum = instance[key]['token_sum']
            layer_res[idx]['cosine_sum'] += one_cosine_sum
            layer_res[idx]['token_sum'] += one_token_sum
            
    # Compute the final result for each layer and store it in a list
    res_list = []
    for idx in range(1,13):
        res_list.append(round(layer_res[idx]['cosine_sum'] / layer_res[idx]['token_sum'], 3))
    return res_list


In [ ]:

def generategraphs():
    # Load the results of self-similarity analysis for BERT and TaCL models
    in_f = r'./bert_resul.json'
    bert_res_list = process_result(in_f)
    in_f = r'./tacl_resul.json'
    tacl_res_list = process_result(in_f)

    # Import required libraries for plotting the graph
    import matplotlib.pyplot as plt

    # Configure the graph style
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    plt.rcParams.update({'font.size': 14})
    plt.xlabel('Layer', fontsize=16)
    plt.ylabel('Self-similarity', fontsize=16)

    # Define x-axis values (layer numbers)
    x = [1,2,3,4,5,6,7,8,9,10,11,12]

    # Define y-axis values (self-similarity scores) for BERT and TaCL models
    y3 = bert_res_list
    y4 = tacl_res_list

    # Define labels for x-axis ticks
    my_xticks = ['1', '2','3','4','5','6','7','8','9','10', '11', '12']

    # Define colors for the graph
    color_1, color_2 = 'orangered', 'royalblue'

    # Configure the x-axis ticks
    plt.xticks(x, my_xticks, fontsize=14)

    # Plot the self-similarity scores for TaCL and BERT models
    plt.plot(x, y4, marker = 's', markerfacecolor='none', label='TaCL(Chinese)', linestyle='--', color=color_1)
    plt.plot(x, y3, marker = 'd', markerfacecolor='none', linestyle='--', label='BERT(Chinese)', color=color_2)

    # Configure the graph limits and legend
    plt.ylim(ymin=0.1, ymax=0.77)
    plt.legend(loc='upper left', fontsize=12)
    plt.legend(fontsize=12, ncol=2, handleheight=2, labelspacing=0.005, columnspacing=0.5, loc='upper left')

    # Display and save the graph
    plt.show()
    plt.savefig('self-similarity.png', format='png', dpi=500, bbox_inches = 'tight')
